# Create a simple tensor with random items

In [3]:
import numpy as np 

# Supress scientific notation
np.set_printoptions(suppress=True)

# Generate randomly distributed parameters
params = np.random.uniform(low=-50, high=150, size = 20)

# Make sure important values are at the beginning for better debugging
params[0] = np.max(params) + 1
params[1] = np.min(params) - 1
params[2] = 0

# Roound each number to the second decimal place
params = np.round(params, 2)

print(params)

[144.85 -47.43   0.   -20.5   33.63 104.61  66.07 -42.76  78.97 130.62
 113.44 -46.43 143.74  13.96  55.25  83.21 105.47 107.85 143.85  79.95]


# Define Quantization methods and quantize

In [4]:
def clamp(params_q: np.array, lower_bound: int, upper_bound: int) -> np.array:
    params_q[params_q < lower_bound] = lower_bound
    params_q[params_q > upper_bound] = upper_bound
    return params_q

def asymmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float, int]:
    # calculate scale and zero point
    alpha = np.max(params)
    beta = np.min(params)
    scale = (alpha - beta) / (2**bits - 1)
    zero = -1*np.round(beta/scale)
    lower_bound, upper_bound = (0, 2**bits-1)

    # Quantize the parameters
    quantized = clamp(np.round(params/scale + zero), lower_bound, upper_bound).astype(np.int32)
    return quantized, scale, zero

def asymmetric_dequantize(params_q: np.array, scale: float, zero: int):
    return scale*(params_q - zero)

def symmetric_quantization(params: np.array, bits: int) -> tuple[np.array, float]:
    # calculate scale
    alpha = np.max(np.abs(params))
    scale = np.abs(alpha) / (2**(bits-1) - 1)
    lower_bound = -2**(bits-1) -1
    upper_bound = 2**(bits-1) -1

    # Quantize the parameters
    quantized = clamp(np.round(params/scale), lower_bound, upper_bound)
    return quantized, scale

def symmetric_dequantize(params_q: np.array, scale: float):
    return scale * params_q

def quantization_error(params: np.array, params_q: np.array):
    # Calculate the MSE
    return np.mean((params - params_q)**2)


# Quantize to 8 bits

In [5]:
(asymmetric_q, asymmetric_scale, asymmetric_zero) = asymmetric_quantization(params, 8)
(symmetric_q, symmetric_scale) = symmetric_quantization(params, 8)

print(f"Original: {np.round(params, 2)}")
print(f"Asymmetric scale: {asymmetric_scale}, zero: {asymmetric_zero}")
print(asymmetric_q)
print(f"Symmetric scale: {symmetric_scale}")
print(symmetric_q)

Original: [144.85 -47.43   0.   -20.5   33.63 104.61  66.07 -42.76  78.97 130.62
 113.44 -46.43 143.74  13.96  55.25  83.21 105.47 107.85 143.85  79.95]
Asymmetric scale: 0.7540392156862745, zero: 63.0
[255   0  63  36 108 202 151   6 168 236 213   1 254  82 136 173 203 206
 254 169]
Symmetric scale: 1.140551181102362
[127. -42.   0. -18.  29.  92.  58. -37.  69. 115.  99. -41. 126.  12.
  48.  73.  92.  95. 126.  70.]


# Dequantize to 32 bits

In [6]:
params_deq_asymmetric = asymmetric_dequantize(asymmetric_q, asymmetric_scale, asymmetric_zero)
params_deq_symmetric = symmetric_dequantize(symmetric_q, symmetric_scale)

print(f"Original: {np.round(params, 2)}")
print(f"Dequantize Asymmetric: {params_deq_asymmetric}")
print("")
print(f"Dequantize Symmetric: {params_deq_symmetric}")


Original: [144.85 -47.43   0.   -20.5   33.63 104.61  66.07 -42.76  78.97 130.62
 113.44 -46.43 143.74  13.96  55.25  83.21 105.47 107.85 143.85  79.95]
Dequantize Asymmetric: [144.77552941 -47.50447059   0.         -20.35905882  33.93176471
 104.81145098  66.35545098 -42.98023529  79.17411765 130.44878431
 113.10588235 -46.75043137 144.0214902   14.3267451   55.04486275
  82.94431373 105.5654902  107.82760784 144.0214902   79.92815686]

Dequantize Symmetric: [144.85       -47.90314961   0.         -20.52992126  33.07598425
 104.93070866  66.1519685  -42.2003937   78.6980315  131.16338583
 112.91456693 -46.76259843 143.70944882  13.68661417  54.74645669
  83.26023622 104.93070866 108.3523622  143.70944882  79.83858268]


# Calculate quantization error

In [8]:
print(f"Asymmetric Error: {np.round(quantization_error(params,params_deq_asymmetric), 2)}")
print(f"symmetric Error: {np.round(quantization_error(params, params_deq_symmetric), 2)}")


Asymmetric Error: 0.05
symmetric Error: 0.13


Bad pipe message: %s [b'"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";']
Bad pipe message: %s [b'"120"\r\nsec-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Wi', b'ows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36\r\nAccept: text/']
Bad pipe message: %s [b'ml,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/s']
Bad pipe message: %s [b'ol: max-age=0\r\nsec-ch-ua: "Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"\r\nsec-ch-ua-mobile: ?0\r\ns']
Bad pipe message: %s [b'-ch-ua-platform: "Windows"\r\nUpgrade-Insecure-Requests: 1\r\nUser-Agent: Mozilla/5.0 (Windows NT 10.0; ', b'n64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36\r\nAccep', b' text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signe']
